# **Pattern Recognition and Machine Learning**
> Week 6 Tutorial

## **Tutorial 1 - Text Classification**

## Classification of news articles into different topics using Naive Bayes
### 1. Retrieve data

In [ ]:
from sklearn.datasets import fetch_20newsgroups

news_groups = fetch_20newsgroups(subset='all')

### 2. Explore dataset

In [ ]:
news_groups.data[1]

In [ ]:
list(news_groups.target_names)

In [ ]:
news_groups.filenames.shape

In [ ]:
news_groups.target.shape

### 3. Calculate words frequency
Use CountVectorizer to calculate a bag of words and their frequencies.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

An example first:

In [ ]:
document = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

In [ ]:
cv.fit_transform(document)
cv.vocabulary_

In [ ]:
cv.fit_transform(document).toarray()

Output a bag of words and their frequency.

In [ ]:
word_list = cv.get_feature_names()
count_list = cv.fit_transform(document).toarray().sum(axis=0)
print(dict(zip(word_list,count_list)))

### 4. Extract features

In [ ]:
features = cv.fit_transform(news_groups.data)

In [ ]:
features.shape

In [ ]:
features.toarray()

### 5. Split into train/test sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features,
                                                    news_groups.target,
                                                    train_size=0.8,
                                                    random_state=11)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

### 6. Build multinomil Naive Bayes model and fit

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB(alpha=1)
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

In [ ]:
print(y_test)
print(y_pred)

### 7. Evaluate the model

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred, average='weighted'))
print("Recall:",metrics.recall_score(y_test, y_pred, average='weighted'))
print("F1-score:",metrics.f1_score(y_test, y_pred, average='weighted'))

In [ ]:
report = classification_report(y_test, y_pred,
                               target_names=news_groups.target_names)
print(report)

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

### 8. Find the optimal alpha using GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

nb = MultinomialNB()

Parameters setting for alpha: 10, 1, 0.1, 0.001, 0.0001

In [ ]:
params = {'alpha': [10, 1, 1e-1, 1e-2, 1e-3]}

Run the Grid Search and fit the data

In [ ]:
# 10-fold
#grs = GridSearchCV(nb, param_grid=params, cv = 10)

# 5-fold default
grs = GridSearchCV(nb, param_grid=params)

grs.fit(X_train, y_train)

The optimal value

In [ ]:
print("Best Hyper Parameters:",grs.best_params_)

### 9. Evaluate the result

In [ ]:
y_pred=grs.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred, average='weighted'))
print("Recall:",metrics.recall_score(y_test, y_pred, average='weighted'))
print("F1-score:",metrics.f1_score(y_test, y_pred, average='weighted'))

## **Tutorial 2 - Ransomware Detection using K-NN**
### 1. Retrieve data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

bitcoin_heist = pd.read_csv("data/BitcoinHeistData.csv")

### 2. Explore data

In [ ]:
bitcoin_heist.shape

In [ ]:
bitcoin_heist.info()

In [ ]:
bitcoin_heist.head(3)

In [ ]:
bitcoin_heist.describe()

In [ ]:
bitcoin_heist.describe(include="O")

In [ ]:
bitcoin_heist.dtypes

### 3. Convert categorical values to numerical values

In [ ]:
bitcoin_heist['labels'] = [0 if x == 'white' else 1 for x in bitcoin_heist['label']]

bitcoin_heist['labels'].value_counts()

### 4. Extract features

In [ ]:
X = bitcoin_heist.loc[0:200000, ['year','day', 'length', 'weight', 'count',
                                 'looped', 'neighbors', 'income']]
y = bitcoin_heist.loc[0:200000, 'labels']

### 5. Split data into train/test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    train_size=0.8,
                                                    random_state=11)

### 6. Build and fit K-NN model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

### 7. Evaluate the model

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
f1_score(y_test, y_pred, average='weighted')

### 8. Tune parameter using GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()
params = {'n_neighbors': range(1,10)}

# 10-fold
#grs = GridSearchCV(knn, param_grid=params, cv = 10)

# 5-fold default
grs = GridSearchCV(knn, param_grid=params)
grs.fit(X_train, y_train)

In [ ]:
print("Best Hyper Parameters:",grs.best_params_)

In [ ]:
from sklearn import metrics

y_pred=grs.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred, average='weighted'))
print("Recall:",metrics.recall_score(y_test, y_pred, average='weighted'))
print("F1-score:",metrics.f1_score(y_test, y_pred, average='weighted'))

Put more data into training.

In [ ]:
X = bitcoin_heist[['year','day', 'length', 'weight', 'count',
                  'looped', 'neighbors', 'income']]
y = bitcoin_heist['labels']

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    train_size=0.8,
                                                    random_state=11)

knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

confusion_matrix(y_test, y_pred)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred, average='weighted'))
print("Recall:",metrics.recall_score(y_test, y_pred, average='weighted'))
print("F1-score:",metrics.f1_score(y_test, y_pred, average='weighted'))